In [1]:
# 10/2/2021 - Building A Quantitative Value Investing Strategy

# Algorithmic Trading Using Python - Full Course
# https://www.youtube.com/watch?v=xfzGZB4HhEE
# https://iexcloud.io/docs/api/

# https://github.com/nickmccullum/algorithmic-trading-python

import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [70]:
stocks = pd.read_csv('sp_500_stocks.csv')
#stocks['Ticker']
from secrets import IEX_CLOUD_API_TOKEN

In [71]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(api_url)
# print(data.status_code)
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 89137216, 'calculationPrice': 'close', 'change': 1.21, 'changePercent': 0.00817, 'close': 149.09, 'closeSource': 'oifcailf', 'closeTime': 1643864260970, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 144.58, 'delayedPriceTime': 1635281204047, 'extendedChange': -0.07, 'extendedChangePercent': -0.0005, 'extendedPrice': 149.12, 'extendedPriceTime': 1702484317658, 'high': 149.09, 'highSource': ' iuel p1reeinydcea5d mt', 'highTime': 1679597644093, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 143.03, 'iexCloseTime': 1709342613236, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 148.36, 'iexOpenTime': 1684642032787, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1664673829269, 'latestPrice': 146.12, 'latestSource': 'Close', 'latestTime': 'October 1, 2021', 'latestUpdate': 1634900830824, 'latestVolume': 98680561, 'low': 144.08, 'lowSource': 'ycdldt 1erae nimp

In [72]:
price = data['latestPrice']
pe_ratio = data['peRatio']

In [73]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

#for symbol_string in symbol_strings:
#    print(symbol_string)

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [74]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#    print(data.status_code)
    for symbol in symbol_string.split(','):
#        pass
#        print(symbol)
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,163.40,48.6,N/A
1,AAL,21.68,-2.22,N/A
2,AAP,209.00,22.66,N/A
3,AAPL,144.93,28.56,N/A
4,ABBV,111.00,30.5,N/A
...,...,...,...,...
500,YUM,126.32,29.78,N/A
501,ZBH,149.99,34.76,N/A
502,ZBRA,533.95,37.74,N/A
503,ZION,63.97,6.5,N/A


In [75]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
# final_dataframe
# len(final_dataframe)
# final_dataframe.reset_index()
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,71.81,3.8,N/A
1,MPC,63.97,5.43,N/A
2,BIO,769.61,5.62,N/A
3,PRU,110.57,6.07,N/A
4,GM,54.52,6.23,N/A
5,ZION,63.97,6.5,N/A
6,AFL,53.59,6.62,N/A
7,COF,166.95,6.76,N/A
8,CINF,117.32,6.86,N/A
9,WHR,215.26,7.03,N/A


In [76]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

In [79]:
portfolio_input()

Enter the size of your portfolio:1000000


In [155]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,71.81,3.8,696
1,MPC,63.97,5.43,781
2,BIO,769.61,5.62,64
3,PRU,110.57,6.07,452
4,GM,54.52,6.23,917
5,ZION,63.97,6.5,781
6,AFL,53.59,6.62,933
7,COF,166.95,6.76,299
8,CINF,117.32,6.86,426
9,WHR,215.26,7.03,232


In [156]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# data.status_code
print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
# print(data[symbol]['quote']['peRatio'])
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise value divided by Earning Before Interest, Taxes, Depreciation and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit


{'beta': None, 'totalCash': 62260995158, 'currentDebt': 109458177591, 'revenue': 362487191959, 'grossProfit': 143802613229, 'totalRevenue': 354512368360, 'EBITDA': 113657373556, 'revenuePerShare': 21, 'revenuePerEmployee': 2410555.03, 'debtToEquity': 5.274216961441945, 'profitMargin': 0.2509773748285344, 'enterpriseValue': 2515008336711, 'enterpriseValueToRevenue': 7.1, 'priceToSales': 7.05, 'priceToBook': 38.44, 'forwardPERatio': 25.65325165686877, 'pegRatio': 0.5164606973488716, 'peHigh': 17.068113387547676, 'peLow': 11.11015962155395, 'week52highDate': '2021-09-01', 'week52lowDate': '2020-10-26', 'putCallRatio': 0.4730267250111143, 'companyName': 'Apple Inc', 'marketcap': 2441809923059, 'week52high': 157.88, 'week52low': 107.2, 'week52highSplitAdjustOnly': 160.21, 'week52highDateSplitAdjustOnly': '2021-09-07', 'week52lowSplitAdjustOnly': 112.68, 'week52lowDateSplitAdjustOnly': '2020-10-28', 'week52change': 0.2737260759652412, 'sharesOutstanding': 17113338519, 'float': None, 'avg10Vo

In [157]:
# rv = robust value

rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
#    data = requests.get(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'            
        ],
        index = rv_columns),
            ignore_index = True
        )
    

In [158]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.62,N/A,49.1,N/A,9.77,N/A,7.82,N/A,30.872881,N/A,15.038310,N/A,N/A
1,AAL,22.17,N/A,-2.32,N/A,-1.9,N/A,0.7586,N/A,-4.698549,N/A,1.908171,N/A,N/A
2,AAP,218.00,N/A,22.62,N/A,4.06,N/A,1.2,N/A,11.460099,N/A,2.724559,N/A,N/A
3,AAPL,145.77,N/A,28.37,N/A,37.12,N/A,6.87,N/A,22.521541,N/A,16.762465,N/A,N/A
4,ABBV,110.88,N/A,30.7,N/A,15.41,N/A,3.77,N/A,10.479217,N/A,7.398149,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.51,N/A,28.53,N/A,-4.86,N/A,5.88,N/A,21.528241,N/A,10.031299,N/A,N/A
501,ZBH,152.38,N/A,35.67,N/A,2.49,N/A,3.97,N/A,20.594542,N/A,6.739288,N/A,N/A
502,ZBRA,522.38,N/A,37.02,N/A,10.8,N/A,5.41,N/A,27.540834,N/A,11.782483,N/A,N/A
503,ZION,65.51,N/A,6.4,N/A,1.39,N/A,3.04,N/A,5.111917,N/A,2.684293,N/A,N/A


In [159]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

#len(rv_dataframe[rv_dataframe.isnull().any(axis = 1)].index)

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,188.400,N/A,60.46,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,294.600,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,288.340,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.83,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.620,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.550,N/A,14.9,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.270,N/A,32.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,38.730,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2844.670,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,106.890,N/A,34.12,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [160]:
# for column in rv_dataframe.columns:
#    print(column)
#    rv_dataframe[column].fillna(rv_dataframe[column].mean())

In [161]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [162]:
for column in {'Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP'}:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [163]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [201]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100

rv_dataframe

C:\Users\unpat\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.81,1864,7.58,0.38,0.5006,0.14,0.3968,0.08,3.435678,0.16,0.390335,0.04,4.594059
1,PRU,109.27,457,5.90,0.2,0.6515,0.18,0.6540,0.26,4.678175,0.26,0.642145,0.06,5.267327
2,AAL,22.17,2255,-2.32,0.18,-1.9000,0.12,0.7586,0.32,-4.698549,0.06,1.908171,0.5,6.178218
3,MCK,207.09,241,-7.22,0.14,-60.7600,0.04,0.1263,0.02,7.990669,0.88,2.819321,0.8,7.920792
4,KSS,49.09,1018,10.70,0.64,1.4000,0.58,0.3964,0.06,4.156389,0.18,1.152458,0.24,8.039604
5,L,55.31,903,9.75,0.54,0.7951,0.26,0.9717,0.42,5.536257,0.42,0.983951,0.2,8.158416
6,RE,254.78,196,7.87,0.44,0.9793,0.32,0.8994,0.4,6.768534,0.64,0.923202,0.18,8.435644
7,LNC,72.78,687,10.00,0.58,0.6144,0.16,0.6891,0.28,7.495120,0.76,0.655733,0.08,8.673267
8,BA,232.00,215,-14.92,0.1,-8.1600,0.1,2.2000,0.96,-32.137496,0.02,-225.157479,0.02,8.930693
9,HPQ,28.17,1774,9.08,0.5,-8.5500,0.08,0.5462,0.2,6.632663,0.56,2.813048,0.78,9.148515


In [202]:
from statistics import mean

# for row in rv_dataframe.index[:1]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
#    print(value_percentiles)

rv_dataframe


C:\Users\unpat\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.81,1864,7.58,0.38,0.5006,0.14,0.3968,0.08,3.435678,0.16,0.390335,0.04,0.16
1,PRU,109.27,457,5.90,0.2,0.6515,0.18,0.6540,0.26,4.678175,0.26,0.642145,0.06,0.192
2,AAL,22.17,2255,-2.32,0.18,-1.9000,0.12,0.7586,0.32,-4.698549,0.06,1.908171,0.5,0.236
3,MCK,207.09,241,-7.22,0.14,-60.7600,0.04,0.1263,0.02,7.990669,0.88,2.819321,0.8,0.376
4,KSS,49.09,1018,10.70,0.64,1.4000,0.58,0.3964,0.06,4.156389,0.18,1.152458,0.24,0.34
5,L,55.31,903,9.75,0.54,0.7951,0.26,0.9717,0.42,5.536257,0.42,0.983951,0.2,0.368
6,RE,254.78,196,7.87,0.44,0.9793,0.32,0.8994,0.4,6.768534,0.64,0.923202,0.18,0.396
7,LNC,72.78,687,10.00,0.58,0.6144,0.16,0.6891,0.28,7.495120,0.76,0.655733,0.08,0.372
8,BA,232.00,215,-14.92,0.1,-8.1600,0.1,2.2000,0.96,-32.137496,0.02,-225.157479,0.02,0.24
9,HPQ,28.17,1774,9.08,0.5,-8.5500,0.08,0.5462,0.2,6.632663,0.56,2.813048,0.78,0.424


In [203]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe

C:\Users\unpat\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.81,1864,7.58,0.38,0.5006,0.14,0.3968,0.08,3.435678,0.16,0.390335,0.04,0.16
1,PRU,109.27,457,5.90,0.2,0.6515,0.18,0.6540,0.26,4.678175,0.26,0.642145,0.06,0.192
2,AAL,22.17,2255,-2.32,0.18,-1.9000,0.12,0.7586,0.32,-4.698549,0.06,1.908171,0.5,0.236
8,BA,232.00,215,-14.92,0.1,-8.1600,0.1,2.2000,0.96,-32.137496,0.02,-225.157479,0.02,0.24
4,KSS,49.09,1018,10.70,0.64,1.4000,0.58,0.3964,0.06,4.156389,0.18,1.152458,0.24,0.34
5,L,55.31,903,9.75,0.54,0.7951,0.26,0.9717,0.42,5.536257,0.42,0.983951,0.2,0.368
7,LNC,72.78,687,10.00,0.58,0.6144,0.16,0.6891,0.28,7.495120,0.76,0.655733,0.08,0.372
3,MCK,207.09,241,-7.22,0.14,-60.7600,0.04,0.1263,0.02,7.990669,0.88,2.819321,0.8,0.376
11,ALL,133.60,374,10.48,0.6,1.4900,0.68,0.8104,0.38,2.492573,0.14,0.776833,0.1,0.38
6,RE,254.78,196,7.87,0.44,0.9793,0.32,0.8994,0.4,6.768534,0.64,0.923202,0.18,0.396


In [204]:
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.81,1864,7.58,0.38,0.5006,0.14,0.3968,0.08,3.435678,0.16,0.390335,0.04,0.16
1,PRU,109.27,457,5.90,0.2,0.6515,0.18,0.6540,0.26,4.678175,0.26,0.642145,0.06,0.192
2,AAL,22.17,2255,-2.32,0.18,-1.9000,0.12,0.7586,0.32,-4.698549,0.06,1.908171,0.5,0.236
3,BA,232.00,215,-14.92,0.1,-8.1600,0.1,2.2000,0.96,-32.137496,0.02,-225.157479,0.02,0.24
4,KSS,49.09,1018,10.70,0.64,1.4000,0.58,0.3964,0.06,4.156389,0.18,1.152458,0.24,0.34
5,L,55.31,903,9.75,0.54,0.7951,0.26,0.9717,0.42,5.536257,0.42,0.983951,0.2,0.368
6,LNC,72.78,687,10.00,0.58,0.6144,0.16,0.6891,0.28,7.495120,0.76,0.655733,0.08,0.372
7,MCK,207.09,241,-7.22,0.14,-60.7600,0.04,0.1263,0.02,7.990669,0.88,2.819321,0.8,0.376
8,ALL,133.60,374,10.48,0.6,1.4900,0.68,0.8104,0.38,2.492573,0.14,0.776833,0.1,0.38
9,RE,254.78,196,7.87,0.44,0.9793,0.32,0.8994,0.4,6.768534,0.64,0.923202,0.18,0.396


In [205]:
portfolio_input()

Enter the size of your portfolio:2500000


In [206]:
portfolio_size
position_size = float(portfolio_size) / len(rv_dataframe.index)
position_size


50000.0

In [207]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
    
rv_dataframe
    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.81,1864,7.58,0.38,0.5006,0.14,0.3968,0.08,3.435678,0.16,0.390335,0.04,0.16
1,PRU,109.27,457,5.90,0.2,0.6515,0.18,0.6540,0.26,4.678175,0.26,0.642145,0.06,0.192
2,AAL,22.17,2255,-2.32,0.18,-1.9000,0.12,0.7586,0.32,-4.698549,0.06,1.908171,0.5,0.236
3,BA,232.00,215,-14.92,0.1,-8.1600,0.1,2.2000,0.96,-32.137496,0.02,-225.157479,0.02,0.24
4,KSS,49.09,1018,10.70,0.64,1.4000,0.58,0.3964,0.06,4.156389,0.18,1.152458,0.24,0.34
5,L,55.31,903,9.75,0.54,0.7951,0.26,0.9717,0.42,5.536257,0.42,0.983951,0.2,0.368
6,LNC,72.78,687,10.00,0.58,0.6144,0.16,0.6891,0.28,7.495120,0.76,0.655733,0.08,0.372
7,MCK,207.09,241,-7.22,0.14,-60.7600,0.04,0.1263,0.02,7.990669,0.88,2.819321,0.8,0.376
8,ALL,133.60,374,10.48,0.6,1.4900,0.68,0.8104,0.38,2.492573,0.14,0.776833,0.1,0.38
9,RE,254.78,196,7.87,0.44,0.9793,0.32,0.8994,0.4,6.768534,0.64,0.923202,0.18,0.396


In [214]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)


In [215]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [216]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile', percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [217]:
writer.save()